# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

In [ ]:
import os
import json
from testing_utilities import write_json_to_file
%load_ext dotenv

## Setup

Below are the various name definitions for the resources needed to setup AKS.

In [ ]:
%%writefile --append .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# If you have multiple subscriptions select the subscription you want to use 
selected_subscription = "YOUR_SUBSCRIPTION"

# Resource group, name and location for AKS cluster.
resource_group = "RESOURCE_GROUP" 
aks_name = "AKS_CLUSTER_NAME"
location = "eastus"

In [ ]:
%dotenv
image_name = os.getenv('docker_login') + os.getenv('image_repo')

## Azure account login

If you are not already logged in to an Azure account, the command below will initiate a login. This will pop up a browser where you can select your login.

In [ ]:
%%bash
list=`az account list -o table`
if [ \"$list\" == '[]' ] || [ \"$list\" == '' ]; then
  az login -o table
else
  az account list -o table
fi

In [ ]:
!az account set --subscription "$selected_subscription"

In [ ]:
!az account show

You will also need to register the container service resources on your subscription if you haven't already done so.

In [ ]:
!az provider register -n Microsoft.ContainerService

In [ ]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [ ]:
!az group create --name $resource_group --location $location

Below, we create the AKS cluster in the resource group we created earlier. This could take up to 15 minutes.

In [ ]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [ ]:
!sudo env "PATH=$PATH" az aks install-cli

## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [ ]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Let's verify connection by listing the nodes.

In [ ]:
!kubectl get nodes

Let's check the pods on our cluster.

In [ ]:
!kubectl get pods --all-namespaces

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [ ]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": image_name,
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "mountPath":"/usr/local/nvidia",
                              "name": "nvidia",
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [ ]:
write_json_to_file(app_template, 'az-dl.json')

In [ ]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [ ]:
!cat az-dl.json

Next, we will use kubectl create command to deploy our application.

In [ ]:
!kubectl create -f az-dl.json

Let's check if the pod is deployed. It may take as many as 10 minutes for the container to be ready.

In [ ]:
!kubectl get pods --all-namespaces

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [ ]:
!kubectl get events

Check the logs for the application pod.

In [ ]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [ ]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

In [ ]:
!kubectl get deployment

It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [ ]:
!kubectl get service azure-dl

Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb).